In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
from nltk.stem import WordNetLemmatizer
import nltk
%matplotlib inline

In [2]:
data = pd.read_csv("/Users/Joe/Desktop/Fall2018/Data401/Project2/Data/beer.csv")

In [3]:
lemmatizer = WordNetLemmatizer()
stopwords = set(nltk.corpus.stopwords.words('english'))

In [4]:
def processSentence(sentence):
    sentence = sentence.lower()
    for char in ":\";'?!@#$%^&*(){}[]\<>,.":
        sentence = sentence.replace(char, "")
        
    sentence = sentence.split()
    
    sentence = [lemmatizer.lemmatize(word) for word in sentence]
    
    sentence = [word for word in sentence if word not in stopwords]
    
    return sentence

In [5]:
data['processedText'] = data['review/text'].fillna("").map(processSentence)

In [6]:
vectorDimension = 100
wordModel = Word2Vec(sentences=data['processedText'], size=vectorDimension, window=9, min_count=20, workers=-1, sg=0)

In [7]:
from collections import defaultdict

In [8]:
wordCountDict = defaultdict(int)
for sentence in data['processedText']:
    for word in np.unique(sentence):
        wordCountDict[word] += 1

tfidfDict = {}
N = len(data['processedText'])
for word in wordCountDict.keys():
    tfidfDict[word] = np.log(N / wordCountDict[word])

In [9]:
def tryGetVector(word):
    try:
        return wordModel.wv.get_vector(word)
    except:
        return np.zeros(vectorDimension)
def vectorizeSentence(sentence):
    vectors = np.array([tryGetVector(word) * tfidfDict[word] for word in sentence])
    return np.sum(vectors, axis = 0)

In [10]:
data['wordVector'] = data['processedText'].map(vectorizeSentence)

In [11]:
from gensim.test.utils import common_texts

In [12]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument